In [58]:
import cohere
import requests
from edgar import *
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup
import os
import json
import datetime
from pprint import pprint
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import uuid
import hnswlib
from typing import List, Dict
from unstructured.partition.html import partition_html
from unstructured.chunking.title import chunk_by_title

# COHERE_API_KEY = "c2guXlp1mohZNOKnEpYiFyOtsQNHeU5L99j6JkGF"

co = cohere.Client(COHERE_API_KEY,
                   log_warning_experimental_features=False,    
                   )

In [59]:
def get_stock_ticker_and_range(prompt):
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    
    ticker = co.chat(
        model="command-r-plus",
        preamble="you are going to return only the ticker for the company that the user is asking about. The ticker should be formatted with MAX 4 characters and MIN 2 characters",
        message=prompt,
        connectors=[{"id": "web-search"}]
    )
    
    pprint(ticker.text)
    
    response = co.chat(
    model="command-r-plus",
    preamble=f"I want you to generate a JSON that represents a query that the user made about a company's stock with ticker, formatted with MAX 4 characters and MIN 2 characters, and the start and end date of the query formatted as YYYY-MM-DD. Today is going to be the date {today} if there is no end date known, do the last year.",
    message=f"{prompt} this information will help you get the ticker: {ticker.text}", 
    response_format={
            "type": "json_object",
            "schema": {
                "type": "object",
                "required": ["tickers", "start-date", "end-date"],
                "properties": {
                    "tickers": { "type": "array" },
                    "start-date": { "type": "string"},
                    "end-date": { "type": "string" }
                }
            }
        },
    # connectors= [{"id": "web-search"}]
    )
    
    response = json.loads(response.text)
    
    return response["tickers"], response["start-date"], response["end-date"]

In [60]:
get_stock_ticker_and_range("What is the stock ticker for Apple?")

'AAPL'


(['AAPL'], '2024-09-28', '2023-09-28')

In [61]:
def get_stock_data(prompt):
    
    ticker, start_date, end_date = get_stock_ticker_and_range(prompt)
    loaded_data = yf.download(ticker, start=start_date, end=end_date)

    return loaded_data

In [65]:
raw_documents = [
    {
        "title": "Crafting Effective Prompts",
        "url": "https://docs.cohere.com/docs/crafting-effective-prompts"},
    {
        "title": "Advanced Prompt Engineering Techniques",
        "url": "https://docs.cohere.com/docs/advanced-prompt-engineering-techniques"},
    {
        "title": "Batch embedding jobs",
        "url": "https://docs.cohere.com/docs/embed-jobs-api"},
    {
        "title": "Preambles",
        "url": "https://docs.cohere.com/docs/preambles"}
]

In [66]:
class Vectorstore:
    """
    A class representing a collection of documents indexed into a vectorstore.

    Parameters:
    raw_documents (list): A list of dictionaries representing the sources of the raw documents. Each dictionary should have 'title' and 'url' keys.

    Attributes:
    raw_documents (list): A list of dictionaries representing the raw documents.
    docs (list): A list of dictionaries representing the chunked documents, with 'title', 'text', and 'url' keys.
    docs_embs (list): A list of the associated embeddings for the document chunks.
    docs_len (int): The number of document chunks in the collection.
    idx (hnswlib.Index): The index used for document retrieval.

    Methods:
    load_and_chunk(): Loads the data from the sources and partitions the HTML content into chunks.
    embed(): Embeds the document chunks using the Cohere API.
    index(): Indexes the document chunks for efficient retrieval.
    retrieve(): Retrieves document chunks based on the given query.
    """

    def __init__(self, raw_documents: List[Dict[str, str]]):
        self.raw_documents = raw_documents
        self.docs = []
        self.docs_embs = []
        self.retrieve_top_k = 10
        self.rerank_top_k = 3
        self.load_and_chunk()
        self.embed()
        self.index()


    def load_and_chunk(self) -> None:
        """
        Loads the text from the sources and chunks the HTML content in parallel.
        """
        print("Loading documents...")
        
        def process_document(raw_document):
            try:
                elements = partition_html(url=raw_document["url"], headers={
                    "User-Agent": "ks@gatech.edu"
                })
                chunks = chunk_by_title(elements)
                return [
                    {
                        "title": raw_document["title"],
                        "text": str(chunk),
                        "url": raw_document["url"],
                    }
                    for chunk in chunks
                ]
            except Exception as e:
                print(f"Error loading document: {e}")
                return []

        # Use ThreadPoolExecutor for parallel processing
        with ThreadPoolExecutor() as executor:
            # Submit all tasks
            future_to_doc = {executor.submit(process_document, doc): doc for doc in self.raw_documents}
            
            # Process results as they complete
            for future in tqdm(as_completed(future_to_doc), total=len(self.raw_documents), desc="Processing documents"):
                self.docs.extend(future.result())

    def embed(self) -> None:
        """
        Embeds the document chunks using the Cohere API.
        """
        print("Embedding document chunks...")

        batch_size = 90
        self.docs_len = len(self.docs)
        for i in range(0, self.docs_len, batch_size):
            if i % 100 == 0:
                print(f"Processing document chunk {i} of {self.docs_len}...")
            batch = self.docs[i : min(i + batch_size, self.docs_len)]
            texts = [item["text"] for item in batch]
            docs_embs_batch = co.embed(
                texts=texts, model="embed-english-v3.0", input_type="search_document"
            ).embeddings
            self.docs_embs.extend(docs_embs_batch)
        
        
    def index(self) -> None:
        """
        Indexes the document chunks for efficient retrieval.
        """
        print("Indexing document chunks...")

        self.idx = hnswlib.Index(space="ip", dim=1024)
        self.idx.init_index(max_elements=self.docs_len, ef_construction=512, M=64)
        self.idx.add_items(self.docs_embs, list(range(len(self.docs_embs))))

        print(f"Indexing complete with {self.idx.get_current_count()} document chunks.")

    def retrieve(self, query: str) -> List[Dict[str, str]]:
        """
        Retrieves document chunks based on the given query.

        Parameters:
        query (str): The query to retrieve document chunks for.

        Returns:
        List[Dict[str, str]]: A list of dictionaries representing the retrieved document chunks, with 'title', 'text', and 'url' keys.
        """

        # Dense retrieval
        query_emb = co.embed(
            texts=[query], model="embed-english-v3.0", input_type="search_query"
        ).embeddings
        
        doc_ids = self.idx.knn_query(query_emb, k=self.retrieve_top_k)[0][0]

        # Reranking
        rank_fields = ["title", "text"] # We'll use the title and text fields for reranking

        docs_to_rerank = [self.docs[doc_id] for doc_id in doc_ids]
        rerank_results = co.rerank(
            query=query,
            documents=docs_to_rerank,
            top_n=self.rerank_top_k,
            model="rerank-english-v3.0",
            rank_fields=rank_fields
        )

        doc_ids_reranked = [doc_ids[result.index] for result in rerank_results.results]

        docs_retrieved = []
        for doc_id in doc_ids_reranked:
            docs_retrieved.append(
                {
                    "title": self.docs[doc_id]["title"],
                    "text": self.docs[doc_id]["text"],
                    "url": self.docs[doc_id]["url"],
                }
            )

        return docs_retrieved

In [67]:
vectorstore = Vectorstore(raw_documents)

Loading documents...
Embedding document chunks...
Indexing document chunks...
Indexing complete with 172 document chunks.


In [68]:
def run_chatbot(message, chat_history=[]):
    
    # Generate search queries, if any        
    response = co.chat(message=message,
                        model="command-r-plus",
                        search_queries_only=True,
                        chat_history=chat_history)
    
    search_queries = []
    for query in response.search_queries:
        search_queries.append(query.text)

    # If there are search queries, retrieve the documents
    if search_queries:
        print("Retrieving information...", end="")

        # Retrieve document chunks for each query
        documents = []
        for query in search_queries:
            documents.extend(vectorstore.retrieve(query))

        # Use document chunks to respond
        response = co.chat_stream(
            message=message,
            model="command-r-plus",
            documents=documents,
            chat_history=chat_history,
        )

    else:
        response = co.chat_stream(
            message=message,
            model="command-r-plus",
            chat_history=chat_history,
        )
        
    # Print the chatbot response and citations
    chatbot_response = ""
    print("\nChatbot:")

    for event in response:
        if event.event_type == "text-generation":
            print(event.text, end="")
            chatbot_response += event.text
        if event.event_type == "stream-end":
            if event.response.citations:
                print("\n\nCITATIONS:")
                for citation in event.response.citations:
                    print(citation)
            if event.response.documents:
                print("\nCITED DOCUMENTS:")
                for document in event.response.documents:
                    print(document)
            # Update the chat history for the next turn
            chat_history = event.response.chat_history

    return chat_history

In [70]:
vectorstore.retrieve("Prompting by giving examples")

[{'title': 'Advanced Prompt Engineering Techniques',
  'text': 'Few-shot Prompting\n\nUnlike the zero-shot examples above, few-shot prompting is a technique that provides a model with examples of the task being performed before asking the specific question to be answered. We can steer the LLM toward a high-quality solution by providing a few relevant and diverse examples in the prompt. Good examples condition the model to the expected response type and style.',
  'url': 'https://docs.cohere.com/docs/advanced-prompt-engineering-techniques'},
 {'title': 'Crafting Effective Prompts',
  'text': 'Incorporating Example Outputs\n\nLLMs respond well when they have specific examples to work from. For example, instead of asking for the salient points of the text and using bullet points “where appropriate”, give an example of what the output should look like.',
  'url': 'https://docs.cohere.com/docs/crafting-effective-prompts'},
 {'title': 'Advanced Prompt Engineering Techniques',
  'text': 'In a

In [72]:
chat_history = run_chatbot("Hello, I have a question")


Chatbot:
Of course! I'm here to help. Please go ahead with your question, and I'll do my best to assist you.

In [73]:
chat_history = run_chatbot("What's the difference between zero-shot and few-shot prompting", chat_history)

Retrieving information...
Chatbot:
Zero-shot prompting is when a model is asked to perform a task without being given any examples of how to do it. On the other hand, few-shot prompting is a technique where a model is provided with a few examples of the task being performed before being asked to answer a specific question. These examples help steer the model toward a high-quality solution by conditioning it to the expected response type and style.

CITATIONS:
start=0 end=19 text='Zero-shot prompting' document_ids=['doc_0', 'doc_3']
start=63 end=95 text='without being given any examples' document_ids=['doc_0', 'doc_3']
start=132 end=150 text='few-shot prompting' document_ids=['doc_0', 'doc_3']
start=199 end=239 text='few examples of the task being performed' document_ids=['doc_0', 'doc_3']
start=310 end=356 text='steer the model toward a high-quality solution' document_ids=['doc_0', 'doc_3']
start=360 end=416 text='conditioning it to the expected response type and style.' document_ids=[

In [74]:
chat_history = run_chatbot("How would the latter help?", chat_history)

Retrieving information...
Chatbot:
Few-shot prompting can vastly improve the quality of a model's completions. By providing a few relevant and diverse examples, the model can be steered toward a high-quality solution by conditioning it to the expected response type and style.

CITATIONS:
start=23 end=75 text="vastly improve the quality of a model's completions." document_ids=['doc_2']
start=95 end=124 text='relevant and diverse examples' document_ids=['doc_0']
start=143 end=181 text='steered toward a high-quality solution' document_ids=['doc_0']
start=185 end=241 text='conditioning it to the expected response type and style.' document_ids=['doc_0']

CITED DOCUMENTS:
{'id': 'doc_2', 'text': 'Advanced Prompt Engineering Techniques\n\nThe previous chapter discussed general rules and heuristics to follow for successfully prompting the Command family of models. Here, we will discuss specific advanced prompt engineering techniques that can in many cases vastly improve the quality of the mode